<a href="https://colab.research.google.com/github/shirin1704/Python-Repository/blob/main/Traffic%20Lights%20Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob

Enter 6 months PALMS report in 'palms'<br>
Enter 6 months training report in 'training'<br>
**needs to be updated every month**

In [ ]:
palms = pd.read_csv("/content/6 Months Palms October 24.csv")
training = pd.read_csv("/content/Training Report October 24.csv")

Extracting useful columns from PALMS report into a dataframe<br>
Calculating number of weeks<br>
Calculating total referrals given<br>
Renaming columns for basic formatting

In [ ]:
palms_columns = ['Chapter', 'First Name','Last Name', 'P', 'A', 'L', 'M', 'S', 'RGI', 'RGO', 'V', 'TYFCB (INR)', 'T']
palms_processed = pd.DataFrame(palms[palms_columns])
palms_processed["Weeks"] = palms_processed['P'] + palms_processed['A'] + palms_processed['L'] + palms_processed['M'] + palms_processed['S']
palms_processed["Referrals Given"] = palms_processed['RGI'] + palms_processed['RGO']
palms_processed.rename(columns = {'V':'Visitors', 'T':'Testimonials'}, inplace = True)
palms_processed

Extracting useful columns from training report into a dataframe<br>
Creating a new column that combines chapter name, first name and last name so joining dataframes becomes easier<br>
Counting number of trainings for each member and renaming the column for basic formatting

In [ ]:
training_columns = ['Chapter Name', 'First Name', 'Last Name']
training_processed = pd.DataFrame(training[training_columns])
training_processed["Name"] = training_processed['Chapter Name'] + training_processed['First Name'] + training_processed['Last Name']
training_processed = training_processed.Name.value_counts().reset_index()
training_processed.rename(columns = {'count' : 'Training Count'}, inplace = True)
training_processed

Creating a new column in the PALMS report that combines chapter name, first name and last name so joining the dataframes becomes easier

In [ ]:
palms_processed["Name"] = palms_processed['Chapter'] + palms_processed['First Name'] + palms_processed['Last Name']
palms_processed

Joining the dataframes based on the new Name column created using an outer join so values that do not find a match are automatically assigned a null value<br>
Replacing the null values in the training count column with the value 0

In [ ]:
final_file = palms_processed.merge(training_processed, on = 'Name', how = 'outer')
final_file = final_file.drop('Name', axis = 1)
final_file['Training Count'] = final_file['Training Count'].fillna(0)
final_file

Calculating the score for Absenteeism

In [ ]:
final_file['Absent Score'] = 0
final_file.loc[final_file['A'] > 2, 'Absent Score'] = 0
final_file.loc[final_file['A'] == 2, 'Absent Score'] = 5
final_file.loc[final_file['A'] == 1, 'Absent Score'] = 10
final_file.loc[final_file['A'] < 1, 'Absent Score'] = 15
final_file

Calculating the score for Referrals Given per Week

In [ ]:
final_file['Referrals/Week'] = final_file['Referrals Given']/final_file['Weeks']
final_file['Referrals Score'] = 0
final_file.loc[final_file['Referrals/Week'] < 0.5, 'Referrals Score'] = 0
final_file.loc[(final_file['Referrals/Week'] < 0.75) & (final_file['Referrals/Week'] >= 0.5), 'Referrals Score'] = 5
final_file.loc[(final_file['Referrals/Week'] < 1) & (final_file['Referrals/Week'] >= 0.75), 'Referrals Score'] = 10
final_file.loc[(final_file['Referrals/Week'] < 1.2) & (final_file['Referrals/Week'] >= 1), 'Referrals Score'] = 15
final_file.loc[final_file['Referrals/Week'] >= 1.2, 'Referrals Score'] = 20
final_file

Calculating the score for TYFTB

In [ ]:
final_file['TYFTB Score'] = 0
final_file.loc[final_file['TYFCB (INR)'] < 500000, 'TYFTB Score'] = 0
final_file.loc[(final_file['TYFCB (INR)'] < 1000000) & (final_file['TYFCB (INR)'] >= 500000), 'TYFTB Score'] = 5
final_file.loc[(final_file['TYFCB (INR)'] < 2000000) & (final_file['TYFCB (INR)'] >= 1000000), 'TYFTB Score'] = 10
final_file.loc[final_file['TYFCB (INR)'] >= 2000000, 'TYFTB Score'] = 15
final_file

Calculating the score for Visitors per Week

In [ ]:
final_file['Visitors/Week'] = final_file['Visitors']/final_file['Weeks']
final_file['Visitors Score'] = 0
final_file.loc[final_file['Visitors/Week'] < 0.1, 'Visitors Score'] = 0
final_file.loc[(final_file['Visitors/Week'] < 0.25) & (final_file['Visitors/Week'] >= 0.1), 'Visitors Score'] = 5
final_file.loc[(final_file['Visitors/Week'] < 0.5) & (final_file['Visitors/Week'] >= 0.25), 'Visitors Score'] = 10
final_file.loc[(final_file['Visitors/Week'] < 0.75) & (final_file['Visitors/Week'] >= 0.5), 'Visitors Score'] = 15
final_file.loc[final_file['Visitors/Week'] >= 0.75, 'Visitors Score'] = 20
final_file

Calculating the score for Testimonials per Week

In [ ]:
final_file['Testimonials/Week'] = final_file['Testimonials']/final_file['Weeks']
final_file['Testimonials Score'] = 0
final_file.loc[final_file['Testimonials/Week'] <= 0, 'Testimonials Score'] = 0
final_file.loc[(final_file['Testimonials/Week'] < 0.075) & (final_file['Testimonials/Week'] > 0), 'Visitors Score'] = 5
final_file.loc[final_file['Testimonials/Week'] >= 0.075, 'Testimonials Score'] = 10
final_file

Calculating the score for Arriving on Time

In [ ]:
final_file['Arriving on Time Score'] = 5
final_file.loc[final_file['L'] >= 1, 'Arriving on Time Score'] = 0

Calculating the score for Training

In [ ]:
final_file['Training Score'] = 0
final_file.loc[final_file['Training Count'] == 0, 'Training Score'] = 0
final_file.loc[final_file['Training Count'] == 1, 'Training Score'] = 5
final_file.loc[final_file['Training Count'] == 2, 'Training Score'] = 10
final_file.loc[final_file['Training Count'] >= 3, 'Training Score'] = 15

Extracting only the column names of the final file

In [ ]:
final_file.columns

Creating a report dataframe that includes each member's following details:
<li>Chapter Name</li>
<li>First Name</li>
<li>Last Name</li>
<li>Absent Score</li>
<li>Referrals Score</li>
<li>TYFTB Score</li>
<li>Visitors Score</li>
<li>Testimonials Score</li>
<li>Arriving on Time Score</li>
<li>Training Score</li>

Generating a Total score by summing all the scores



In [ ]:
name_columns = ['Chapter', 'First Name', 'Last Name']
score_columns = ['Absent Score', 'Referrals Score', 'TYFTB Score', 'Visitors Score', 'Testimonials Score', 'Arriving on Time Score', 'Training Score']
report_columns = name_columns + score_columns
report = pd.DataFrame(final_file[report_columns])
report['Total'] = 0
for score in score_columns:
  report['Total'] = report['Total'] + report[score]
report

Exporting the report dataframe for the given month into an excel sheet<br>
**needs to be changed every month**

In [ ]:
report.to_excel('October 2024.xlsx')